# Exploratory Data Analysis - E-commerce Fraud Data

This notebook performs exploratory data analysis on the e-commerce transaction data (Fraud_Data.csv).


## 1. Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## 2. Load Data


In [ ]:
# Load the dataset
data_path = Path('../data/raw/Fraud_Data.csv')
df = pd.read_csv(data_path)

print(f"Dataset shape: {df.shape}")
df.head()
